# Baseline v3.1 - Unsloth 최적화 + 최종 학습 버전

**주요 변경 사항:**
- **K-Fold 제거**: 최적 하이퍼파라미터 탐색 단계 이후, 전체 학습 데이터로 단일 학습을 진행.
- **전체 데이터 사용**: `train.csv`의 모든 데이터를 학습에 사용합니다 (샘플링 제거).
- **Wandb 연동**: 스텝 수에 대해 학습 과정을 연속적으로 모니터링하기 위해 `wandb`를 사용.
- **상세 프롬프트 적용**: Few-Shot 예시와 CoT가 적용된 상세 시스템 프롬프트를 사용.
- **LoRA 설정 업데이트**: `r=32`, `lora_alpha=32`, `lora_dropout=0.05`, `use_rslora=True` 등 K-Fold 실험에서 사용된 설정 적용.
- **학습 설정 업데이트**: `num_train_epochs=5`, `cosine` 스케줄러, `packing=True` 등을 적용.

Colab의 GPU 환경(T4 GPU)에서 개발되었습니다.
- **런타임 > 런타임 유형 변경 > T4 GPU 혹은 A100과 고용량 RAM사용**으로 설정.

# 1. 환경 준비

Unsloth, Wandb 및 최신 라이브러리 설치 코드

In [2]:
%%capture
# colab & local PC용 라이브러리 설치
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.57.0
!pip install --no-deps trl==0.22.2
!pip install wandb scikit-learn albumentations # 추가 라이브러리

# 2. 데이터 준비 및 Wandb 로그인

구글 드라이브를 마운트하고 대회 데이터를 압축한 후, Wandb에 로그인

In [ ]:
# 구글드라이브 마운트
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# 데이터 압축 해제 (Colab 경로 예시, 환경에 맞게 수정)
!unzip -q "/home/team100/251024/data.zip" -d "/home/team100/content/" # 경로 확인 필요

In [3]:
import wandb
wandb.login() # API 키 입력

/home/team100/.venv/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/team100/.venv/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias

True

# 3. 라이브러리, 데이터, 설정 로드

In [2]:
import os, random
import pandas as pd
from PIL import Image
import torch
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split # To split data
from unsloth import FastVisionModel
from trl import SFTTrainer, SFTConfig
from unsloth.chat_templates import get_chat_template
from unsloth.trainer import UnslothVisionDataCollator
from tqdm import tqdm
import math

# 이미지 로드 시 픽셀 제한 해제
Image.MAX_IMAGE_PIXELS = None

# 디바이스 GPU 우선 사용 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# 설정값 정의
MODEL_ID = "unsloth/Qwen3-VL-8B-Instruct-unsloth-bnb-4bit" # Unsloth의 8B 4bit 양자화 모델
MAX_SEQ_LENGTH = 2048
MAX_NEW_TOKENS = 8
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# 데이터셋 로드 (전체 데이터 사용)
train_df = pd.read_csv("/home/team100/train.csv")
test_df  = pd.read_csv("/home/team100/test.csv")

# train_df = train_df.sample(n=200, random_state=SEED).reset_index(drop=True) # 샘플링 제거

Device: cuda


# 4. Unsloth 모델 및 LoRA 어댑터 로딩

Unsloth의 `FastVisionModel`을 사용하여 4bit 양자화된 모델과 토크나이저를 로드하고, 업데이트된 LoRA 설정을 적용.

In [5]:
model, tokenizer = FastVisionModel.from_pretrained(
    model_name = MODEL_ID,
    max_seq_length = MAX_SEQ_LENGTH,
    load_in_4bit = True,
    dtype = torch.bfloat16, # Added for potential performance improvement
    fast_inference = False,
    gpu_memory_utilization = 0.8,
)

# LoRA 어댑터 추가 (업데이트된 설정)
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True,
    finetune_language_layers   = True,
    finetune_attention_modules = True,
    finetune_mlp_modules       = True,

    r = 32,           # Increased from 16
    lora_alpha = 32,  # Increased from 16
    lora_dropout = 0.05,# Added dropout
    bias = "none",
    random_state = SEED,
    use_rslora = True,  # Enabled Rank Stabilized LoRA
    loftq_config = None,
    use_gradient_checkpointing = "unsloth",
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"], # Explicitly target modules
)

model.print_trainable_parameters()

==((====))==  Unsloth 2025.10.9: Fast Qwen3_Vl patching. Transformers: 4.57.0.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.494 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.01s/it]
Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


trainable params: 87,293,952 || all params: 8,854,417,648 || trainable%: 0.9859


In [6]:
from trl.trainer.sft_trainer import DataCollatorForVisionLanguageModeling
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained(MODEL_ID)
collator = DataCollatorForVisionLanguageModeling(processor)

# 5. 프롬프트 템플릿 및 데이터 포맷팅

상세화된 시스템 프롬프트를 사용하여 데이터를 모델 형식에 맞게 변환하고, 훈련/검증 데이터로 분할합니다.

In [7]:
# Pandas DataFrame을 Hugging Face Dataset으로 변환
raw_dataset = Dataset.from_pandas(train_df)

# Resize to (512, 512) and handle image loading errors
def convert_to_rgb(example):
    try:
        example["decoded_image"] = Image.open(example["path"]).resize((512, 512)).convert("RGB")
    except Exception as e:
        print(f"Error loading image {example['path']}: {e}")
        # Create a white dummy image
        example["decoded_image"] = Image.new("RGB", (512, 512), color = 'white')
    return example

raw_dataset = raw_dataset.map(convert_to_rgb, num_proc=os.cpu_count()) # Use multiple processes for faster mapping

Map (num_proc=12):   0%|          | 0/3887 [00:00<?, ? examples/s]

Map (num_proc=12): 100%|██████████| 3887/3887 [00:57<00:00, 67.18 examples/s] 


In [ ]:
# 상세화된 시스템 프롬프트 (Few-Shot 예시 포함)
SYSTEM_INSTRUCT = ("""
<|im_start|>system
당신은 논리적인 추론가이자 시각 분석 전문가입니다.

각 이미지와 질문에 대해, 다음 3단계 사고 과정을 거쳐 답변을 생성하세요.
1.  **관찰:** 이미지에서 질문과 관련된 핵심적인 시각 정보(사물, 텍스트, 상황 등)를 객관적으로 서술합니다.
2.  **추론:** 관찰한 내용을 바탕으로, 질문의 의도에 맞춰 어떤 보기가 정답인지 논리적인 과정을 단계별로 설명합니다.
3.  **최종 답변:** 모든 추론을 종합하여, `최종 답변: [알파벳]` 형식으로 명확하게 결론을 내립니다.

---
**Few-Shot 예시**

**예시 1:** 원형 표지판에 “40 km/h”가 적혀 있는 도시 도로
질문: "이 도시의 제한 속도는 얼마인가요?"
보기: (a) 40 km/h (b) 60 km/h (c) 80 km/h (d) 100 km/h
**관찰:** 이미지 중앙에 원형 교통 표지판이 있으며, 그 안에 '40 km/h'라는 텍스트가 명확하게 보입니다.
**추론:** 질문은 이 지역의 '제한 속도'를 묻고 있습니다. 표지판의 '40 km/h'는 '시속 40킬로미터'를 의미하며, 이는 차량의 최대 허용 속도를 나타내는 교통 규제입니다. 따라서 보기 a가 표지판의 내용과 일치합니다.
**최종 답변: a**

**예시 2:** 기념비 옆에 쌓여 있는 돌탑들
질문: "이 사진에서 볼 수 있는 돌탑들의 주요 목적은 무엇인가요?"
보기: (a) 기념과 소원을 위한 돌탑 쌓기 (b) 돌을 분류하기 위한 작업 (c) 예술 작품 전시를 위한 조형물 (d) 건축 자재로 사용하기 위한 돌 쌓기
**관찰:** 여러 크기의 돌들이 사람이 인위적으로 쌓아 올린 탑 형태를 이루고 있습니다. 주변 배경은 자연 또는 기념 장소로 보입니다.
**추론:** 한국 문화에서 이러한 돌탑은 주로 소원을 빌거나 무언가를 기념하기 위해 쌓는 민속적인 행위입니다. 다른 보기인 분류, 예술, 건축 자재 등은 일반적인 돌탑의 목적과 거리가 멉니다. 따라서 보기 a가 가장 적절한 설명입니다.
**최종 답변: a**

**예시 3:** 거리의 자동판매기
질문: "이 이미지에서 볼 수 있는 장치의 주된 용도는 무엇인가요?"
보기: (a) 물품을 보관하는 무인 보관함 (b) 자동 판매기 (c) 음식을 조리하는 전자레인지 (d) 인터넷 검색용 키오스크
**관찰:** 이미지에는 유리창 너머로 캔이나 병과 같은 다양한 음료가 진열된 기계가 보입니다. 돈이나 카드를 넣는 투입구와 하단에 물건이 나오는 배출구가 있습니다.
**추론:** 이 장치는 사람이 없이 상품을 진열하고 결제 시스템을 통해 자동으로 판매하도록 설계되었습니다. 이는 '자동 판매기'의 정의와 정확히 일치합니다. 다른 보기인 보관함, 전자레인지, 키오스크는 시각적 증거와 맞지 않습니다.
**최종 답변: b**

**예시 4:** “No Parking”이라고 적힌 도로 표지판
질문: "이 표지판이 전달하는 의미는 무엇인가요?"
보기: (a) 주차 가능 구역 (b) 주차 금지 구역 (c) 보행자 전용 구역 (d) 일방통행 구역
**관찰:** 이미지에는 "No Parking"이라는 텍스트가 명확하게 적힌 표준 교통 표지판이 있습니다.
**추론:** 질문은 표지판의 의미를 묻고 있습니다. "No Parking"이라는 문구는 해당 구역에 차량을 주차하는 것을 명백히 금지한다는 뜻입니다. 보기 b '주차 금지 구역'은 이 의미를 직접적으로 설명합니다.
**최종 답변:** b

**예시 5:** 카페 메뉴판에 “카푸치노 4.5, 아메리카노 4.0”이 적혀 있음
질문: "이 이미지의 텍스트가 나타내는 정보는 무엇인가요?"
보기: (a) 카페의 영업 시간 (b) 음료 메뉴와 가격 정보 (c) 카페 위치 정보 (d) 할인 쿠폰 안내
**관찰:** 텍스트는 "카푸치노", "아메리카노"와 같은 음료 이름과 그 옆에 "4.5", "4.0"과 같은 숫자를 나열하고 있습니다.
**추론:** 음료 이름과 숫자를 함께 나열하는 것은 카페 메뉴판에서 가격을 표시하는 표준적인 형식입니다. 따라서 이 텍스트는 음료 메뉴와 그 가격 정보를 나타냅니다. 보기 b가 이 내용과 정확히 일치합니다.
**최종 답변:** b
<|im_end|>
""")


def build_mc_prompt(question, a, b, c, d):
    return (
        f"{question}\n"
        f"(a) {a}\n(b) {b}\n(c) {c}\n(d) {d}\n\n"
        "정답을 반드시 a, b, c, d 중 하나의 소문자 한 글자로만 출력하세요."
    )

# 학습 데이터를 모델의 대화 형식(messages)으로 변환하는 함수
def make_conversation(example):
    user_text = build_mc_prompt(str(example["question"]), str(example["a"]), str(example["b"]), str(example["c"]), str(example["d"]))
    gold = str(example["answer"]).strip().lower()

    messages = [
        {"role":"system","content":[{"type":"text","text":SYSTEM_INSTRUCT}]},
        {
            "role":"user",
            "content":[
                {"type":"image"}, # Placeholder for the image
                {"type":"text","text":user_text}  # The text part of the prompt
                ]
         },
          {
              "role":"assistant",
              "content":[
                  {"type":"text","text":gold}
                  ]
            }
    ]
    # The actual image data is kept separate for the processor
    # Check if 'decoded_image' exists before trying to access it
    image_data = [example["decoded_image"]] if "decoded_image" in example else []
    return {"messages": messages, "images": image_data}

# Apply conversation formatting
dataset = raw_dataset.map(
    make_conversation,
    remove_columns=["path", "a", "b", "c", "d", "question", "id", "answer", "decoded_image"],
    num_proc=os.cpu_count() # Use multiple processes
)

# 훈련/검증 데이터 분리 (전체 포맷팅된 데이터 대상)
dataset_split = dataset.train_test_split(test_size=0.1, seed=SEED)
train_dataset = dataset_split["train"]
valid_dataset = dataset_split["test"]

print("Training dataset size:", len(train_dataset))
print("Validation dataset size:", len(valid_dataset))
print("Final training sample:\n", train_dataset[0])

Map (num_proc=12):   0%|          | 0/3887 [00:00<?, ? examples/s]

Map (num_proc=12): 100%|██████████| 3887/3887 [01:07<00:00, 57.25 examples/s] 

Training dataset size: 3498
Validation dataset size: 389
Final training sample:
 {'messages': [{'content': [{'text': '\n당신은 시각적 질문에 답변하는 도우미입니다.\n각 이미지와 질문을 보고, 이미지 안의 사물이나 표지판, 문구의 의미를 이해한 뒤\n제시된 보기(a, b, c, d) 중 하나를 정확히 선택하세요.\n답변은 반드시 \'a\', \'b\', \'c\', \'d\' 중 한 글자만 출력해야 하며, 설명을 덧붙이지 마세요.\n\n이미지 안의 사물이나 텍스트의 의미를 문맥 속에서 해석하세요.\n예를 들어, 속도 제한 표지판이 있다면 그 안의 숫자(예: 40km/h)를 읽고\n이 지역의 제한 속도가 40km/h라는 뜻임을 이해해야 합니다.\n이 경우 정답은 ‘a’입니다.\n\n표지판, 안내문, 메뉴판, 장치 등의 이미지는 그 내용이 전달하는\n구체적인 의미(예: 제한속도, 금지사항, 물건 판매 등)를 파악하여\n이에 맞는 보기를 선택하세요.\n\n------------------------------------\nFew-Shot 예시\n------------------------------------\n\n예시 1\n이미지: 도시 도로에 원형 표지판이 있고 “40”이라고 적혀 있습니다.\n질문: "이 도시의 제한 속도는 얼마인가요?"\n보기:\na. 40 km/h\nb. 60 km/h\nc. 80 km/h\nd. 100 km/h\n정답: a\n이 표지판은 40km/h의 제한 속도를 의미하므로 ‘a’가 정답입니다.\n\n예시 2\n이미지: 기념비 옆에 돌탑들이 쌓여 있습니다.\n질문: "이 사진에서 볼 수 있는 돌탑들의 주요 목적은 무엇인가요?"\n보기:\na. 기념과 소원을 위한 돌탑 쌓기\nb. 돌을 분류하기 위한 작업\nc. 예술 작품 전시를 위한 조형물\nd. 건축 자재로 사용하기 위한 돌 쌓기\n정답: a\n이 돌탑들은 기념비 옆에 쌓은 형태이므로 ‘a’가

# 6. SFTTrainer를 사용한 최종 파인튜닝

전체 학습 데이터를 사용하여 모델 최종 파인튜닝을 진행. 학습 과정은 Wandb를 통해 모니터링되고 출력문 중 View project이후 나오는 링크를 통해 확인 가능.

In [ ]:
# 모델을 학습 모드로 활성화
FastVisionModel.for_training(model)

# 배치 크기 및 스텝 계산
per_device_train_batch_size = 8 # 메모리 부족 방지를 위해 원래 값 유지
gradient_accumulation_steps = 2   # 메모리 부족 방지를 위해 원래 값 유지
effective_batch_size = per_device_train_batch_size * gradient_accumulation_steps
num_train_epochs = 5
total_train_steps = math.ceil((len(train_dataset) * num_train_epochs) / effective_batch_size)
eval_steps = math.ceil(len(train_dataset) / effective_batch_size / 2) # Evaluate twice per epoch
logging_steps = 10 # Log every 10 steps

print(f"Effective Batch Size: {effective_batch_size}")
print(f"Total Training Steps: {total_train_steps}")
print(f"Evaluation Steps: {eval_steps}")

# SFTTrainer 설정 (업데이트된 파라미터)
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    packing=True, # Enable packing
    data_collator=collator,
    args=SFTConfig(
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_train_batch_size * 2, # Can increase eval batch size slightly
        gradient_accumulation_steps=gradient_accumulation_steps,
        warmup_steps=int(total_train_steps * 0.05), # Warmup for 5% of total steps
        num_train_epochs=num_train_epochs,
        learning_rate=1e-4,
        logging_steps=logging_steps,
        eval_strategy="steps",
        eval_steps=eval_steps,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine", # Use cosine scheduler
        seed=SEED,
        output_dir="outputs_final",
        report_to="wandb", # Report to wandb
        run_name="final_train_full_data_8B", # Wandb run name
        remove_unused_columns=False,
        dataset_kwargs={"skip_prepare_dataset": True},
        max_length=MAX_SEQ_LENGTH,
        save_strategy="steps", # Save checkpoints periodically
        save_steps=eval_steps, # Save checkpoint every evaluation step
        save_total_limit=2, # Keep only the last 2 checkpoints
        load_best_model_at_end=True, # Load the best model based on evaluation loss at the end
        metric_for_best_model="eval_loss", # Use eval_loss to determine the best model
        greater_is_better=False, # Lower eval_loss is better
    ),
)

# 학습 시작
print("Starting final training...")
trainer.train()
print("Final training finished.")

# 최종 평가
print("Evaluating the best model...")
final_eval_results = trainer.evaluate()
print("Final Evaluation Results:", final_eval_results)

# Wandb 종료
wandb.finish()

Effective Batch Size: 16
Total Training Steps: 1094
Evaluation Steps: 110


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
The model is already on multiple devices. Skipping the move to device specified in `args`.


Starting final training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,498 | Num Epochs = 5 | Total steps = 1,095
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 2 x 1) = 16
 "-____-"     Trainable parameters = 87,293,952 of 8,854,417,648 (0.99% trained)


wandb: Detected [huggingface_hub.inference] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
110,0.805500,1.610877
220,0.804500,1.608895
330,0.803900,1.608039
440,0.802800,1.607752
550,0.800800,1.608645
660,0.800700,1.608775
770,3.624700,7.094336


Unsloth: Not an error, but Qwen3VLForConditionalGeneration does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


KeyboardInterrupt: 

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x70244c7219f0>> (for post_run_cell), with arguments args (<ExecutionResult object at 70244f1e4700, execution_count=14 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 70244f1e4bb0, raw_cell="# 모델을 학습 모드로 활성화
FastVisionModel.for_training(mode.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bteam100/home/team100/%E1%84%8E%E1%85%AE%E1%84%85%E1%85%A9%E1%86%AB%E1%84%8F%E1%85%A9%E1%84%83%E1%85%B3output%5B0%5D__softmax%E1%84%80%E1%85%B5%E1%84%87%E1%85%A1%E1%86%AB.ipynb#X22sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


# 7. 추론 및 제출 파일 생성

학습된 최종 모델(가장 좋은 성능을 보인 체크포인트)을 사용하여 테스트 데이터에 대한 추론을 수행하고, `submission.csv` 파일을 생성 후 submission으로 제출.

In [9]:
import os
from PIL import Image
import torch
import torch.nn.functional as F
import pandas as pd
from tqdm import tqdm

from unsloth import FastVisionModel
from transformers import LogitsProcessorList

# 0) 모델 로드
CKPT_DIR = "/home/team100/outputs_final/checkpoint-440"
model, tokenizer = FastVisionModel.from_pretrained(
    model_name=CKPT_DIR,
    load_in_4bit=True,
    dtype=torch.float16,   # Kaggle T4 → fp16 권장
    device_map="auto",
)
FastVisionModel.for_inference(model)

# --- (중요) Processor 내부 텍스트 토크나이저 가져오기 헬퍼 ---
def get_text_tokenizer(tkr):
    # 우선순위: .tokenizer > .text_tokenizer > 본인
    if hasattr(tkr, "tokenizer") and tkr.tokenizer is not None:
        return tkr.tokenizer
    if hasattr(tkr, "text_tokenizer") and tkr.text_tokenizer is not None:
        return tkr.text_tokenizer
    return tkr  # fallback (혹시 순수 토크나이저인 경우)

txt_tok = get_text_tokenizer(tokenizer)

# --- 한 글자 선택(a/b/c/d)용 단일 토큰 id들을 최대한 안전하게 모으기 ---
def gather_single_token_ids(txt_tok, ch: str):
    """
    txt_tok.encode로 ch, ' '+ch, 대소문자 변형을 모두 시도하여
    '하나의 토큰'으로 인코딩되는 경우의 id만 모아 반환.
    """
    candidates = [ch, " " + ch, ch.lower(), " " + ch.lower(), ch.upper(), " " + ch.upper()]
    ids = set()
    for s in candidates:
        try:
            toks = txt_tok.encode(s, add_special_tokens=False)
        except TypeError:
            # 일부 토크나이저는 encode 시 키워드 인자 형태만 받기도 함
            toks = txt_tok.encode(s)
        if isinstance(toks, dict) and "input_ids" in toks:
            toks = toks["input_ids"]
        if isinstance(toks, (list, tuple)) and len(toks) == 1:
            ids.add(int(toks[0]))
    return list(ids)

A_IDS = gather_single_token_ids(txt_tok, "a")
B_IDS = gather_single_token_ids(txt_tok, "b")
C_IDS = gather_single_token_ids(txt_tok, "c")
D_IDS = gather_single_token_ids(txt_tok, "d")

# (선택) a/b/c/d 외 토큰 금지
class RestrictToSet:
    def __init__(self, allowed_ids):
        self.allowed = torch.tensor(sorted(list(allowed_ids)))
    def __call__(self, input_ids, scores):
        # scores: [batch, vocab_size]
        mask = scores.new_full(scores.shape, float("-inf"))
        mask[:, self.allowed] = scores[:, self.allowed]
        return mask

USE_VOCAB_RESTRICT = True
allowed_all = set(A_IDS + B_IDS + C_IDS + D_IDS)
logits_processors = LogitsProcessorList([RestrictToSet(allowed_all)]) if (USE_VOCAB_RESTRICT and len(allowed_all) > 0) else None

# 경로/상수
ROOT = "/home/team100/"     # 이미지 루트
SAVE_PATH = "/home/team100/submission_1026.csv"
MAX_NEW_TOKENS = 1

# 추론 루프 (test_df, SYSTEM_INSTRUCT, build_mc_prompt 가 이미 정의돼 있다고 가정)
preds = []
model.eval()
with torch.inference_mode():
    for i in tqdm(range(len(test_df))):
        row = test_df.iloc[i]
        img_path = os.path.join(ROOT, str(row["path"]))
        image = Image.open(img_path).convert("RGB")

        user_text = build_mc_prompt(row["question"], row["a"], row["b"], row["c"], row["d"])

        messages = [
            {"role": "system", "content": [{"type": "text", "text": SYSTEM_INSTRUCT}]},
            {"role": "user", "content": [{"type": "image"}, {"type": "text", "text": user_text}]},
        ]
        prompt = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )

        inputs = tokenizer(
            image,
            prompt,
            add_special_tokens=False,
            return_tensors="pt",
        ).to("cuda")

        gen_kwargs = dict(
            max_new_tokens=MAX_NEW_TOKENS,
            do_sample=False,
            temperature=0.0,
            top_p=1.0,
            use_cache=True,
            return_dict_in_generate=True,
            output_scores=True,
        )
        if logits_processors is not None:
            gen_kwargs["logits_processor"] = logits_processors

        outputs = model.generate(**inputs, **gen_kwargs)

        # 마지막 토큰 확률분포
        scores = outputs.scores[-1][0]   # [vocab_size]
        probs = F.softmax(scores, dim=-1)

        # 각 보기 확률 = 해당 보기의 가능한 단일토큰 id들의 확률 합
        def prob_of(ids):
            if not ids:
                return -1.0  # 안전장치 (토큰이 전혀 못 모였을 경우)
            return float(probs[ids].sum().item())

        p_a = prob_of(A_IDS)
        p_b = prob_of(B_IDS)
        p_c = prob_of(C_IDS)
        p_d = prob_of(D_IDS)

        probs_dict = {"a": p_a, "b": p_b, "c": p_c, "d": p_d}
        pred = max(probs_dict, key=probs_dict.get)

        preds.append({"id": row["id"], "answer": pred})

# 저장
submission = pd.DataFrame(preds, columns=["id", "answer"])
submission.to_csv(SAVE_PATH, index=False)
print(f"Saved {SAVE_PATH}")
print(submission.head())

==((====))==  Unsloth 2025.10.9: Fast Qwen3_Vl patching. Transformers: 4.57.0.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.494 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


INFO:accelerate.utils.modeling: We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
100%|██████████| 3887/3887 [22:02<00:00,  2.94it/s]

Saved /home/team100/submission_1026.csv
          id answer
0  test_0001      b
1  test_0002      b
2  test_0003      b
3  test_0004      c
4  test_0005      c


In [ ]:
# 나의 구글 드라이브 본래 작업 폴더에 저장
# drive_path = "/content/drive/My Drive/251024/submission_final.csv" # 경로 확인 필요, 파일명 변경
# submission.to_csv(drive_path, index=False)
# print(f"Saved to Google Drive: {drive_path}")

In [ ]:
# 최종 학습된 LoRA 어댑터 저장 (옵션)
# model.save_pretrained("final_model_lora")
# tokenizer.save_pretrained("final_model_lora")
# print("Saved LoRA adapter and tokenizer to 'final_model_lora'")